In [1]:
# # app/main.py
# import os
# import time
# import json
# import bisect
# import tempfile
# from datetime import datetime
# from typing import List, Optional, Dict, Any

# import pandas as pd
# from fastapi import FastAPI, HTTPException, BackgroundTasks, Query
# from pydantic import BaseModel, Field

# # External provider functions from nsepython
# # make sure you have `nsepython` installed in your env:
# # pip install nsepython
# from nsepython import option_chain, nse_quote

# # ---------------------------
# # Models
# # ---------------------------

# class IndexPriceResponse(BaseModel):
#     symbol: str
#     lastPrice: float
#     pChange: float
#     change: float
#     timestamp: str

# class StockPriceResponse(BaseModel):
#     symbol: str
#     companyName: Optional[str] = None
#     lastPrice: Optional[float] = None
#     pChange: Optional[float] = None
#     change: Optional[float] = None
#     timestamp: str

# class FetchOptionsRequest(BaseModel):
#     index: str = Field(..., description="Index symbol, e.g. NIFTY or BANKNIFTY")
#     num_strikes: int = Field(25, gt=0, le=500)

# class FetchExpiryRequest(BaseModel):
#     index: str
#     expiry: str
#     num_strikes: int = Field(25, gt=0, le=500)

# class FetchResultMeta(BaseModel):
#     createdAtUTC: str
#     indexName: str
#     nearestExpiry: Optional[str] = None
#     selectedExpiry: Optional[str] = None
#     underlyingValue: Optional[float] = None
#     atmStrike: Optional[int] = None
#     selectedStrikesRange: Optional[List[int]] = None
#     totalStrikesFetched: Optional[int] = None

# class AnalyticsResponse(BaseModel):
#     meta: FetchResultMeta
#     pcr: Dict[str, float]
#     top_oi: Dict[str, List[Dict[str, Any]]]
#     max_pain: Dict[str, Any]

# # ---------------------------
# # App & config
# # ---------------------------

# app = FastAPI(title="Option Chain API", version="1.0",
#               description="Fetch option chains from NSE and return analytics (PCR / MaxPain / Top OI).")

# OUTPUT_DIR = os.environ.get("OPTION_OUTPUT_DIR", "option_chain_data")
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# # ---------------------------
# # Helpers (adapted from your script)
# # ---------------------------

# def _expand_side(df: pd.DataFrame, side: str) -> pd.DataFrame:
#     valid_rows = df[df[side].apply(lambda x: isinstance(x, dict))]
#     if valid_rows.empty:
#         return pd.DataFrame()
#     side_data = valid_rows[side].apply(pd.Series)
#     side_data = side_data.add_prefix(f'{side}_')
#     return side_data

# def _atomic_write_csv(df: pd.DataFrame, target_path: str):
#     # write to temp file then atomically replace
#     dirpath = os.path.dirname(target_path)
#     os.makedirs(dirpath, exist_ok=True)
#     with tempfile.NamedTemporaryFile(mode="w", dir=dirpath, delete=False, suffix=".csv") as tmp:
#         tmp_name = tmp.name
#         df.to_csv(tmp_name, index=False)
#     os.replace(tmp_name, target_path)

# def _atomic_write_json(obj: dict, target_path: str):
#     dirpath = os.path.dirname(target_path)
#     os.makedirs(dirpath, exist_ok=True)
#     with tempfile.NamedTemporaryFile(mode="w", dir=dirpath, delete=False, suffix=".json", encoding="utf-8") as tmp:
#         tmp_name = tmp.name
#         json.dump(obj, tmp, indent=2)
#     os.replace(tmp_name, target_path)

# def _normalize_index_name(index: str) -> str:
#     if not index:
#         return ""
#     s = index.strip().upper()
#     if s in ("NIFTY50", "NIFTY", "NSEI"):
#         return "NIFTY"
#     if s in ("BANKNIFTY", "NSEBANK"):
#         return "BANKNIFTY"
#     return s

# # ---------------------------
# # Analytical helpers (your logic)
# # ---------------------------

# def calculate_pcr(df: pd.DataFrame) -> dict:
#     pcr_data = {'pcr_by_oi': 0.0, 'pcr_by_volume': 0.0}
#     if 'PE_openInterest' in df.columns and 'CE_openInterest' in df.columns:
#         total_pe_oi = df['PE_openInterest'].fillna(0).sum()
#         total_ce_oi = df['CE_openInterest'].fillna(0).sum()
#         if total_ce_oi > 0:
#             pcr_data['pcr_by_oi'] = round(total_pe_oi / total_ce_oi, 2)
#     if 'PE_totalTradedVolume' in df.columns and 'CE_totalTradedVolume' in df.columns:
#         total_pe_volume = df['PE_totalTradedVolume'].fillna(0).sum()
#         total_ce_volume = df['CE_totalTradedVolume'].fillna(0).sum()
#         if total_ce_volume > 0:
#             pcr_data['pcr_by_volume'] = round(total_pe_volume / total_ce_volume, 2)
#     return pcr_data

# def find_high_oi_strikes(df: pd.DataFrame, top_n: int = 5) -> dict:
#     results = {'resistance_strikes': [], 'support_strikes': []}
#     if 'CE_openInterest' in df.columns:
#         top_calls = df.nlargest(top_n, 'CE_openInterest')[['strikePrice', 'CE_openInterest']].fillna(0)
#         results['resistance_strikes'] = top_calls.to_dict('records')
#     if 'PE_openInterest' in df.columns:
#         top_puts = df.nlargest(top_n, 'PE_openInterest')[['strikePrice', 'PE_openInterest']].fillna(0)
#         results['support_strikes'] = top_puts.to_dict('records')
#     return results

# def calculate_max_pain(df: pd.DataFrame) -> dict:
#     if 'strikePrice' not in df.columns:
#         return {'max_pain_strike': None, 'max_loss_value': 0}
#     strikes = sorted(df['strikePrice'].dropna().unique())
#     total_loss_at_strike = {}
#     for strike_price in strikes:
#         loss = 0
#         if 'CE_openInterest' in df.columns and 'CE_lastPrice' in df.columns:
#             ce_data = df[['strikePrice', 'CE_openInterest', 'CE_lastPrice']].dropna()
#             for _, row in ce_data.iterrows():
#                 if row['strikePrice'] > strike_price:
#                     loss += (row['strikePrice'] - strike_price) * row['CE_openInterest']
#         if 'PE_openInterest' in df.columns and 'PE_lastPrice' in df.columns:
#             pe_data = df[['strikePrice', 'PE_openInterest', 'PE_lastPrice']].dropna()
#             for _, row in pe_data.iterrows():
#                 if row['strikePrice'] < strike_price:
#                     loss += (strike_price - row['strikePrice']) * row['PE_openInterest']
#         total_loss_at_strike[strike_price] = loss
#     if not total_loss_at_strike:
#         return {'max_pain_strike': None, 'max_loss_value': 0}
#     max_pain_strike = min(total_loss_at_strike, key=total_loss_at_strike.get)
#     return {'max_pain_strike': int(max_pain_strike), 'max_loss_value': int(total_loss_at_strike[max_pain_strike])}

# # ---------------------------
# # Core fetch + save (refactored)
# # ---------------------------

# def _prepare_option_chain_df(resp: dict, expiry: str) -> pd.DataFrame:
#     if not (isinstance(resp, dict) and 'records' in resp and 'data' in resp['records']):
#         raise RuntimeError("Invalid response structure from NSE.")
#     df_full = pd.DataFrame(resp['records']['data'])
#     if df_full.empty:
#         raise RuntimeError("No option chain data returned by NSE.")
#     if 'strikePrice' not in df_full.columns:
#         raise RuntimeError("Column 'strikePrice' missing from NSE response.")
#     df_full['strikePrice'] = pd.to_numeric(df_full['strikePrice'], errors='coerce')
#     df = df_full[df_full['expiryDate'] == expiry].copy()
#     if df.empty:
#         raise RuntimeError(f"No data for expiry {expiry}")
#     ce_data = _expand_side(df, 'CE')
#     pe_data = _expand_side(df, 'PE')
#     df_processed = pd.concat([df[['strikePrice', 'expiryDate']].reset_index(drop=True), ce_data.reset_index(drop=True), pe_data.reset_index(drop=True)], axis=1)
#     return df_processed

# def _select_strikes_and_save(df_processed: pd.DataFrame, resp: dict, index_name: str, expiry: str, num_strikes: int) -> FetchResultMeta:
#     underlying_value = float(resp['records'].get('underlyingValue', 0))
#     strikes = sorted(df_processed['strikePrice'].dropna().unique())
#     if not strikes:
#         raise RuntimeError("No strikes found after processing")
#     atm_strike_index = bisect.bisect_left(strikes, underlying_value)
#     if atm_strike_index > 0 and abs(strikes[atm_strike_index-1] - underlying_value) < abs(strikes[atm_strike_index] - underlying_value):
#         atm_strike_index -= 1
#     low_index = max(0, atm_strike_index - num_strikes)
#     high_index = min(len(strikes) - 1, atm_strike_index + num_strikes)
#     selected_strikes = strikes[low_index:high_index+1]
#     df_final = df_processed[df_processed['strikePrice'].isin(selected_strikes)].sort_values(['strikePrice']).reset_index(drop=True)
#     timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#     safe_expiry = str(expiry).replace(' ', '_').replace('/', '-')
#     base_filename = f"{index_name.lower()}_option_chain_{safe_expiry}_{timestamp}"
#     csv_path = os.path.join(OUTPUT_DIR, f"{base_filename}.csv")
#     meta_path = os.path.join(OUTPUT_DIR, f"{base_filename}.json")
#     # atomic write
#     _atomic_write_csv(df_final, csv_path)
#     metadata = {
#         'createdAtUTC': datetime.utcnow().isoformat(),
#         'indexName': index_name,
#         'nearestExpiry': expiry,
#         'underlyingValue': float(underlying_value),
#         'atmStrike': int(strikes[atm_strike_index]) if 0 <= atm_strike_index < len(strikes) else None,
#         'selectedStrikesRange': [int(selected_strikes[0]), int(selected_strikes[-1])],
#         'totalStrikesFetched': int(len(df_final))
#     }
#     _atomic_write_json(metadata, meta_path)
#     return FetchResultMeta(**metadata)

# def fetch_and_save_option_chain(index_name: str, num_strikes_around_atm: int = 25) -> FetchResultMeta:
#     start_time = time.time()
#     resp = option_chain(index_name)
#     # find nearest expiry
#     expiries = resp['records'].get('expiryDates', [])
#     if not expiries:
#         raise RuntimeError("No expiries in NSE response.")
#     nearest_expiry = expiries[0]
#     df_processed = _prepare_option_chain_df(resp, nearest_expiry)
#     meta = _select_strikes_and_save(df_processed, resp, index_name, nearest_expiry, num_strikes_around_atm)
#     elapsed = time.time() - start_time
#     app.logger.info(f"Saved option chain for {index_name} expiry {nearest_expiry} in {elapsed:.2f}s")
#     return meta

# def fetch_specific_expiry_option_chain(index_name: str, expiry_date: str, num_strikes_around_atm: int = 25) -> FetchResultMeta:
#     start_time = time.time()
#     resp = option_chain(index_name)
#     expiries = resp['records'].get('expiryDates', [])
#     if expiry_date not in expiries:
#         raise HTTPException(status_code=422, detail=f"Expiry '{expiry_date}' not available. Available: {expiries}")
#     df_processed = _prepare_option_chain_df(resp, expiry_date)
#     meta = _select_strikes_and_save(df_processed, resp, index_name, expiry_date, num_strikes_around_atm)
#     elapsed = time.time() - start_time
#     app.logger.info(f"Saved option chain for {index_name} expiry {expiry_date} in {elapsed:.2f}s")
#     return meta

# # ---------------------------
# # Provider small wrappers
# # ---------------------------

# def get_available_expiries(index_name: str) -> List[str]:
#     try:
#         resp = option_chain(index_name)
#         return resp['records'].get('expiryDates', [])
#     except Exception as e:
#         app.logger.error("get_available_expiries error: %s", e)
#         return []

# def fetch_index_price(index_name: str) -> dict:
#     try:
#         quote = nse_quote(index_name)
#         if not quote or 'lastPrice' not in quote:
#             raise HTTPException(status_code=404, detail=f"No data for index {index_name}")
#         last_price = float(str(quote['lastPrice']).replace(',', ''))
#         return {
#             'symbol': index_name,
#             'lastPrice': last_price,
#             'pChange': float(quote.get('pChange', 0)),
#             'change': float(quote.get('change', 0)),
#             'timestamp': quote.get('secDate', datetime.now().strftime("%d %b %Y %H:%M:%S"))
#         }
#     except HTTPException:
#         raise
#     except Exception as e:
#         app.logger.exception("fetch_index_price error")
#         raise HTTPException(status_code=500, detail=str(e))

# def fetch_stock_price(stock_symbol: str) -> dict:
#     try:
#         quote = nse_quote(stock_symbol)
#         info = quote.get('info', {})
#         price_info = quote.get('priceInfo', {})
#         if not info or not price_info:
#             raise HTTPException(status_code=404, detail=f"No data for stock {stock_symbol}")
#         last_price = price_info.get('lastPrice')
#         try:
#             last_price = float(last_price) if last_price is not None else None
#         except Exception:
#             last_price = None
#         return {
#             'symbol': info.get('symbol'),
#             'companyName': info.get('companyName'),
#             'lastPrice': last_price,
#             'pChange': float(price_info.get('pChange', 0)) if price_info.get('pChange') is not None else None,
#             'change': float(price_info.get('change', 0)) if price_info.get('change') is not None else None,
#             'timestamp': quote.get('metadata', {}).get('lastUpdateTime', datetime.now().strftime("%d-%b-%Y %H:%M:%S"))
#         }
#     except HTTPException:
#         raise
#     except Exception as e:
#         app.logger.exception("fetch_stock_price error")
#         raise HTTPException(status_code=500, detail=str(e))

# # ---------------------------
# # REST endpoints
# # ---------------------------

# @app.get("/expiries", response_model=List[str])
# def api_get_expiries(index: str = Query(..., description="Index symbol, e.g. NIFTY")):
#     idx = _normalize_index_name(index)
#     expiries = get_available_expiries(idx)
#     if not expiries:
#         raise HTTPException(status_code=404, detail=f"No expiries found for {idx}")
#     return expiries

# @app.get("/index-price", response_model=IndexPriceResponse)
# def api_index_price(index: str = Query(..., description="Index symbol, e.g. NIFTY")):
#     idx = _normalize_index_name(index)
#     data = fetch_index_price(idx)
#     return IndexPriceResponse(**data)

# @app.get("/stock-price", response_model=StockPriceResponse)
# def api_stock_price(symbol: str = Query(..., description="Stock symbol (NSE), e.g. RELIANCE")):
#     data = fetch_stock_price(symbol.upper())
#     return StockPriceResponse(**data)

# @app.post("/fetch", response_model=FetchResultMeta, status_code=201)
# def api_fetch_options(request: FetchOptionsRequest, background_tasks: BackgroundTasks):
#     idx = _normalize_index_name(request.index)
#     # we will fetch in background and return immediately: to keep simple, perform sync fetch
#     try:
#         meta = fetch_and_save_option_chain(idx, request.num_strikes)
#         return meta
#     except HTTPException:
#         raise
#     except Exception as e:
#         app.logger.exception("api_fetch_options error")
#         raise HTTPException(status_code=500, detail=str(e))

# @app.post("/fetch/expiry", response_model=FetchResultMeta, status_code=201)
# def api_fetch_options_expiry(req: FetchExpiryRequest):
#     idx = _normalize_index_name(req.index)
#     try:
#         meta = fetch_specific_expiry_option_chain(idx, req.expiry, req.num_strikes)
#         return meta
#     except HTTPException:
#         raise
#     except Exception as e:
#         app.logger.exception("api_fetch_options_expiry error")
#         raise HTTPException(status_code=500, detail=str(e))

# @app.get("/analytics", response_model=AnalyticsResponse)
# def api_analytics_for_latest(index: str = Query(...), limit: int = Query(500, gt=0, le=5000)):
#     """
#     Read the latest saved CSV for the index and compute analytics.
#     """
#     idx = _normalize_index_name(index)
#     files = [f for f in os.listdir(OUTPUT_DIR) if f.startswith(f"{idx.lower()}_") and f.endswith('.csv')]
#     if not files:
#         raise HTTPException(status_code=404, detail=f"No saved option-chain CSVs found for {idx}")
#     latest_file = sorted(files, reverse=True)[0]
#     csv_path = os.path.join(OUTPUT_DIR, latest_file)
#     try:
#         df = pd.read_csv(csv_path)
#     except Exception as e:
#         app.logger.exception("Failed to read CSV for analytics")
#         raise HTTPException(status_code=500, detail="Failed to read saved CSV")
#     # apply limit
#     if limit:
#         df = df.head(limit)
#     pcr = calculate_pcr(df)
#     top_oi = find_high_oi_strikes(df, top_n=5)
#     max_pain = calculate_max_pain(df)
#     # load metadata JSON if present
#     meta_file = csv_path.replace('.csv', '.json')
#     meta_obj = {}
#     if os.path.exists(meta_file):
#         with open(meta_file, 'r', encoding='utf-8') as f:
#             meta_obj = json.load(f)
#     meta_obj.setdefault('createdAtUTC', datetime.utcnow().isoformat())
#     meta = FetchResultMeta(**meta_obj)
#     return AnalyticsResponse(meta=meta, pcr=pcr, top_oi=top_oi, max_pain=max_pain)

# # ---------------------------
# # Simple health endpoint
# # ---------------------------

# @app.get("/health")
# def health():
#     return {"status": "ok", "time": datetime.utcnow().isoformat()}

# # ---------------------------
# # Run example if run directly
# # ---------------------------
# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run("app.main:app", host="127.0.0.1", port=8000, reload=True)


In [2]:
!pip install yfinance pandas requests beautifulsoup4 lxml tqdm nest_asyncio
# Optional (only if you want Crawl4AI fallback and have access)
!pip install crawl4ai


In [11]:
#!/usr/bin/env python3
"""
Final robust fetcher for SENSEX option chain (nearest expiry).
Saves CSV + JSON outputs to ./sensex_option_chain_out/
"""

import json
import re
import time
import traceback
from datetime import datetime
from pathlib import Path

import pandas as pd
import requests
from bs4 import BeautifulSoup

# Optional libs
try:
    import yfinance as yf
    YFINANCE_AVAILABLE = True
except Exception:
    yf = None
    YFINANCE_AVAILABLE = False

# Optional Playwright (sync)
try:
    from playwright.sync_api import sync_playwright, TimeoutError as PlaywrightTimeout
    PLAYWRIGHT_AVAILABLE = True
except Exception:
    sync_playwright = None
    PlaywrightTimeout = Exception
    PLAYWRIGHT_AVAILABLE = False

OUTDIR = Path("./sensex_option_chain_out")
OUTDIR.mkdir(parents=True, exist_ok=True)


def timestamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")


def save_json(obj, fname):
    p = OUTDIR / fname
    with open(p, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)
    print("[saved json]", p)
    return p


def save_csv(df, fname):
    p = OUTDIR / fname
    df.to_csv(p, index=False)
    print("[saved csv ]", p)
    return p


# ---------------- Method A: yfinance ----------------
def attempt_yfinance(symbol="^BSESN"):
    if not YFINANCE_AVAILABLE:
        raise RuntimeError("yfinance not installed")
    print("[attempt] yfinance:", symbol)
    t = yf.Ticker(symbol)
    exps = getattr(t, "options", None)
    if not exps:
        raise RuntimeError("yfinance returned no expiries for symbol")
    nearest = exps[0]
    print("  nearest expiry (yfinance):", nearest)
    oc = t.option_chain(nearest)
    calls = oc.calls.copy(); puts = oc.puts.copy()
    calls["side"] = "CALL"; puts["side"] = "PUT"
    df = pd.concat([calls, puts], ignore_index=True, sort=False)
    meta = {"source": "yfinance", "symbol": symbol, "nearest_expiry": nearest, "retrieved_at": datetime.utcnow().isoformat() + "Z"}
    raw = {"calls": calls.to_dict(orient="records"), "puts": puts.to_dict(orient="records")}
    return {"meta": meta, "df": df, "raw": raw}


# ---------------- Method B: Yahoo JSON with retries ----------------
def attempt_yahoo_json(symbol="^BSESN", max_retries=4):
    print("[attempt] yahoo json endpoint:", symbol)
    base = f"https://query2.finance.yahoo.com/v7/finance/options/{symbol}"
    tries = 0
    while tries < max_retries:
        tries += 1
        try:
            r = requests.get(base, timeout=15)
        except Exception as e:
            wait = 2 ** tries
            print(f"  request error (attempt {tries}/{max_retries}), retry in {wait}s:", e)
            time.sleep(wait)
            continue

        if r.status_code == 200:
            break
        if r.status_code == 429:
            wait = 2 ** tries
            print(f"  429 rate limited by Yahoo (attempt {tries}/{max_retries}), backing off {wait}s")
            time.sleep(wait)
            continue
        else:
            r.raise_for_status()
    else:
        raise RuntimeError("Yahoo JSON failed after retries (429 or network)")

    j = r.json()
    result = j.get("optionChain", {}).get("result")
    if not result:
        raise RuntimeError("Yahoo returned no optionChain result")
    obj = result[0]
    exps = obj.get("expirationDates", [])
    if not exps:
        raise RuntimeError("Yahoo returned no expiries")
    now_ts = int(time.time())
    nearest_ts = next((ts for ts in sorted(exps) if ts >= now_ts), exps[-1])
    print("  nearest expiry (unix):", nearest_ts)

    # fetch explicit expiry
    tries = 0
    while tries < max_retries:
        tries += 1
        r2 = requests.get(base + f"?date={nearest_ts}", timeout=15)
        if r2.status_code == 200:
            break
        if r2.status_code == 429:
            wait = 2 ** tries
            print(f"  expiry fetch 429 (attempt {tries}/{max_retries}), wait {wait}s")
            time.sleep(wait)
            continue
        else:
            r2.raise_for_status()
    else:
        raise RuntimeError("Yahoo expiry fetch failed after retries")

    j2 = r2.json()
    rr = j2.get("optionChain", {}).get("result", [{}])[0]
    options_list = rr.get("options", [])
    if not options_list:
        raise RuntimeError("Yahoo returned empty options list")
    options = options_list[0]
    calls = options.get("calls", []); puts = options.get("puts", [])
    calls_df = pd.DataFrame(calls); calls_df["side"] = "CALL"
    puts_df = pd.DataFrame(puts); puts_df["side"] = "PUT"
    df = pd.concat([calls_df, puts_df], ignore_index=True, sort=False)
    meta = {"source": "yahoo_json", "symbol": symbol, "nearest_expiry_unix": nearest_ts, "retrieved_at": datetime.utcnow().isoformat()+"Z"}
    raw = {"calls": calls, "puts": puts, "full": rr}
    return {"meta": meta, "df": df, "raw": raw}


# ---------------- Method C: Parse saved rendered HTML ----------------
def parse_saved_html(path: Path):
    """
    Attempts to extract option chain info from a saved rendered HTML file.
    - looks for embedded JSON-like blobs (containing option/strike/openInterest/etc)
    - extracts largest tables into CSV
    Returns list of extracted DataFrames (could be empty)
    """
    print("[attempt] parse saved HTML:", path)
    if not path.exists():
        raise RuntimeError(f"Saved HTML file not found: {path}")

    html = path.read_text(encoding="utf-8", errors="ignore")
    soup = BeautifulSoup(html, "lxml")

    extracted_dfs = []
    # 1) extract embedded JSON blobs from scripts
    for s in soup.find_all("script"):
        txt = s.string or ""
        if not txt:
            txt = "".join(s.strings)
        low = txt.lower()
        if any(k in low for k in ("option", "strike", "expiration", "openinterest", "optionchain")):
            for m in re.finditer(r"(\{(?:.|\n){120,200000}\})", txt):
                blob = m.group(1)
                # try strict JSON then tolerant fixes
                try:
                    parsed = json.loads(blob)
                except Exception:
                    cand = blob.replace("'", '"')
                    cand = re.sub(r",\s*}", "}", cand)
                    cand = re.sub(r",\s*]", "]", cand)
                    try:
                        parsed = json.loads(cand)
                    except Exception:
                        continue
                # if parsed a dict with lists of dicts, convert the lists to DataFrame(s)
                if isinstance(parsed, dict):
                    # find lists of dicts (calls/puts/etc)
                    for k, v in parsed.items():
                        if isinstance(v, list) and v and isinstance(v[0], dict):
                            try:
                                df = pd.json_normalize(v)
                                df.attrs["source_key"] = k
                                extracted_dfs.append(df)
                                # save JSON and CSV for debugging
                                fname_base = f"embedded_{k}_{timestamp()}"
                                save_json(v, fname_base + ".json")
                                save_csv(df, fname_base + ".csv")
                            except Exception:
                                pass
    # 2) extract largest HTML tables
    tables = soup.find_all("table")
    if tables:
        def cell_count(t): return sum(len(r.find_all(["td", "th"])) for r in t.find_all("tr"))
        tables_sorted = sorted(tables, key=cell_count, reverse=True)
        for idx, t in enumerate(tables_sorted[:3], 1):
            rows = t.find_all("tr")
            header = None
            data = []
            for i, r in enumerate(rows):
                cells = r.find_all(["th", "td"])
                texts = [c.get_text(separator=" ", strip=True) for c in cells]
                if i == 0 and any(texts):
                    header = texts
                else:
                    if texts and any(tt.strip() for tt in texts):
                        data.append(texts)
            if data:
                maxc = max(len(r) for r in data)
                if not header or len(header) < maxc:
                    header = header or [f"col{i}" for i in range(1, maxc+1)]
                    if len(header) < maxc:
                        header = header + [f"col{i}" for i in range(len(header)+1, maxc+1)]
                rows_norm = [r + [""]*(maxc - len(r)) for r in data]
                df = pd.DataFrame(rows_norm, columns=header)
                fname = f"extracted_table_{idx}_{timestamp()}"
                save_csv(df, fname + ".csv")
                extracted_dfs.append(df)
    return extracted_dfs


# ---------------- Optional Method D: Render page with Playwright sync ----------------
def render_with_playwright_and_extract(urls):
    """
    Uses Playwright sync API to render pages and then parse rendered HTML using same heuristics.
    Requires Playwright + browser binaries installed.
    """
    if not PLAYWRIGHT_AVAILABLE:
        raise RuntimeError("playwright not installed")
    print("[attempt] Playwright rendering (sync) for pages:", urls)
    all_dfs = []
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        for url in urls:
            try:
                page = browser.new_page()
                page.goto(url, wait_until="networkidle", timeout=30000)
                html = page.content()
                # write HTML for debugging
                html_file = OUTDIR / f"playwright_page_{timestamp()}.html"
                html_file.write_text(html, encoding="utf-8")
                print("  saved rendered html:", html_file)
                # parse like saved HTML
                # create soup and reuse parse logic (slightly adapted)
                soup = BeautifulSoup(html, "lxml")
                # reuse logic: extract tables and embedded json
                # reuse parse_saved_html by writing html to temp and calling it
                tmp_file = OUTDIR / f"_tmp_render_{timestamp()}.html"
                tmp_file.write_text(html, encoding="utf-8")
                dfs = parse_saved_html(tmp_file)
                all_dfs.extend(dfs)
            except PlaywrightTimeout as e:
                print("  Playwright timeout for", url, e)
            except Exception as ex:
                print("  Playwright error for", url, ex)
        browser.close()
    return all_dfs


# ---------------- Orchestrator ----------------
def orchestrator(saved_html_path: Path = None):
    symbol = "^BSESN"
    ts = timestamp()

    # 1) Try yfinance
    if YFINANCE_AVAILABLE:
        try:
            out = attempt_yfinance(symbol)
            df = out["df"]; meta = out["meta"]; raw = out["raw"]
            save_csv(df, f"yfinance_optionchain_{ts}.csv")
            save_json(raw, f"yfinance_raw_{ts}.json")
            save_json(meta, f"yfinance_meta_{ts}.json")
            return {"method": "yfinance", "meta": meta, "df": df, "raw": raw}
        except Exception as e:
            print("[yfinance] failed:", e)
            traceback.print_exc()
    else:
        print("yfinance not available — skipping")

    # 2) Try Yahoo JSON
    try:
        out = attempt_yahoo_json(symbol)
        df = out["df"]; meta = out["meta"]; raw = out["raw"]
        save_csv(df, f"yahoo_optionchain_{ts}.csv")
        save_json(raw, f"yahoo_raw_{ts}.json")
        save_json(meta, f"yahoo_meta_{ts}.json")
        return {"method": "yahoo_json", "meta": meta, "df": df, "raw": raw}
    except Exception as e:
        print("[yahoo_json] failed:", e)
        traceback.print_exc()

    # 3) Try parse saved rendered HTML if given or auto-find latest in outdir
    try:
        saved_html = None
        if saved_html_path:
            saved_html = Path(saved_html_path)
        else:
            # find latest crawl4ai / playwright html in OUTDIR
            html_files = sorted(OUTDIR.glob("**/*render*.html"), key=lambda p: p.stat().st_mtime, reverse=True)
            html_files += sorted(OUTDIR.glob("**/crawl4ai_*.html"), key=lambda p: p.stat().st_mtime, reverse=True)
            html_files += sorted(OUTDIR.glob("**/playwright_page_*.html"), key=lambda p: p.stat().st_mtime, reverse=True)
            if html_files:
                saved_html = html_files[0]
        if saved_html and saved_html.exists():
            dfs = parse_saved_html(saved_html)
            if dfs:
                # pick best candidate (largest DF)
                best = max(dfs, key=lambda d: d.shape[0]*d.shape[1])
                save_csv(best, f"parsed_saved_html_optionchain_{ts}.csv")
                save_json({"source": "saved_html", "file": str(saved_html), "retrieved_at": datetime.utcnow().isoformat()+"Z"}, f"parsed_saved_html_meta_{ts}.json")
                return {"method": "parsed_saved_html", "meta": {"file": str(saved_html)}, "df": best, "raw": {"html": str(saved_html)}}
            else:
                print("No extracted tables/json from saved html:", saved_html)
        else:
            print("No saved rendered HTML found to parse.")
    except Exception as e:
        print("[parse_saved_html] failed:", e)
        traceback.print_exc()

    # 4) Optional: try Playwright rendering (if available)
    if PLAYWRIGHT_AVAILABLE:
        try:
            pages = [
                "https://www.bseindia.com/markets/Derivatives/DeriReports/DeriOptionchain.html",
                "https://web.sensibull.com/option-chain?tradingsymbol=SENSEX",
                "https://upstox.com/option-chain/sensex/",
                "https://www.tradingview.com/symbols/BSE-SENSEX/options-chain/",
            ]
            dfs = render_with_playwright_and_extract(pages)
            if dfs:
                best = max(dfs, key=lambda d: d.shape[0]*d.shape[1])
                save_csv(best, f"playwright_extracted_optionchain_{ts}.csv")
                save_json({"source": "playwright_render", "retrieved_at": datetime.utcnow().isoformat()+"Z"}, f"playwright_meta_{ts}.json")
                return {"method": "playwright_render", "meta": {}, "df": best, "raw": {}}
        except Exception as e:
            print("[playwright] failed:", e)
            traceback.print_exc()
    else:
        print("Playwright not available — skipping rendering fallback")

    raise RuntimeError("All automated methods failed. Inspect files in ./sensex_option_chain_out/ for debug HTML/JSON.")


# ---------------- Run if script ----------------
if __name__ == "__main__":
    print("Starting final SENSEX option chain fetch. Output dir:", OUTDIR.resolve())
    try:
        # If you already have a saved combined HTML, pass its path like:
        # result = orchestrator(saved_html_path=Path("sensex_option_chain_out/crawl4ai_all_rendered_20250910_184613.html"))
        result = orchestrator()
        print("Success. Method used:", result.get("method"))
        print("Meta:", result.get("meta"))
        df = result.get("df")
        if isinstance(df, pd.DataFrame):
            print("Preview (top 20 rows):")
            print(df.head(20).to_string())
        else:
            print("No DataFrame returned by selected method.")
    except Exception as exc:
        print("Final failure:", exc)
        traceback.print_exc()
        print("Check folder ./sensex_option_chain_out/ for debug files (html/json).")


Starting final SENSEX option chain fetch. Output dir: D:\WORKSPACE\COMPITITION\STOCK_APIS\FastAPI-Stock-data-\sensex_option_chain_out
[attempt] yfinance: ^BSESN
[yfinance] failed: yfinance returned no expiries for symbol
[attempt] yahoo json endpoint: ^BSESN


Traceback (most recent call last):
  File "C:\Users\omcho\AppData\Local\Temp\ipykernel_19692\1416549496.py", line 277, in orchestrator
    out = attempt_yfinance(symbol)
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\omcho\AppData\Local\Temp\ipykernel_19692\1416549496.py", line 66, in attempt_yfinance
    raise RuntimeError("yfinance returned no expiries for symbol")
RuntimeError: yfinance returned no expiries for symbol


  429 rate limited by Yahoo (attempt 1/4), backing off 2s
  429 rate limited by Yahoo (attempt 2/4), backing off 4s
  429 rate limited by Yahoo (attempt 3/4), backing off 8s
  429 rate limited by Yahoo (attempt 4/4), backing off 16s
[yahoo_json] failed: Yahoo JSON failed after retries (429 or network)
[attempt] parse saved HTML: sensex_option_chain_out\crawl4ai_all_rendered_20250910_184613.html
No extracted tables/json from saved html: sensex_option_chain_out\crawl4ai_all_rendered_20250910_184613.html
[attempt] Playwright rendering (sync) for pages: ['https://www.bseindia.com/markets/Derivatives/DeriReports/DeriOptionchain.html', 'https://web.sensibull.com/option-chain?tradingsymbol=SENSEX', 'https://upstox.com/option-chain/sensex/', 'https://www.tradingview.com/symbols/BSE-SENSEX/options-chain/']
[playwright] failed: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.
Final failure: All automated methods failed. Inspect files in .

Traceback (most recent call last):
  File "C:\Users\omcho\AppData\Local\Temp\ipykernel_19692\1416549496.py", line 291, in orchestrator
    out = attempt_yahoo_json(symbol)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\omcho\AppData\Local\Temp\ipykernel_19692\1416549496.py", line 103, in attempt_yahoo_json
    raise RuntimeError("Yahoo JSON failed after retries (429 or network)")
RuntimeError: Yahoo JSON failed after retries (429 or network)
Traceback (most recent call last):
  File "C:\Users\omcho\AppData\Local\Temp\ipykernel_19692\1416549496.py", line 338, in orchestrator
    dfs = render_with_playwright_and_extract(pages)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\omcho\AppData\Local\Temp\ipykernel_19692\1416549496.py", line 241, in render_with_playwright_and_extract
    with sync_playwright() as p:
  File "d:\WORKSPACE\COMPITITION\STOCK_APIS\.venv\Lib\site-packages\playwright\sync_api\_context_manager.py", line 47, in __enter__
    raise Error(
play

In [10]:
!pip install yfinance pandas requests beautifulsoup4 lxml playwright
# If you want Playwright rendering to work locally, install browser binaries:
!python -m playwright install


^C


In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# Test BSE index data
print('Testing BSE index data...')

# Test SENSEX (^BSESN)
try:
    ticker = yf.Ticker('^BSESN')
    print('SENSEX info available:', bool(ticker.info))
    print('SENSEX current price:', ticker.info.get('regularMarketPrice'))
    print('SENSEX history (last 5 days):')
    hist = ticker.history(period='5d')
    print(hist)
    print()
except Exception as e:
    print('Error with ^BSESN:', e)

# Test BSE Bank Index (BSE-BANK.BO)
try:
    ticker = yf.Ticker('BSE-BANK.BO')
    print('BSE Bank Index info available:', bool(ticker.info))
    print('BSE Bank Index current price:', ticker.info.get('regularMarketPrice'))
    print('BSE Bank Index history (last 5 days):')
    hist = ticker.history(period='5d')
    print(hist)
    print()
except Exception as e:
    print('Error with BSE-BANK.BO:', e)

# Test other BSE indices
bse_indices = ['^BSE100', '^BSE200', '^BSE500']
for symbol in bse_indices:
    try:
        ticker = yf.Ticker(symbol)
        if ticker.info:
            print(f'{symbol} available: {ticker.info.get("regularMarketPrice")}')
        else:
            print(f'{symbol} not available')
    except Exception as e:
        print(f'Error with {symbol}: {e}')

print('\nBSE index data is working successfully!')
hist.to_csv('sensex_history.csv')
print('SENSEX history saved to sensex_history.csv')


Testing BSE index data...
SENSEX info available: True
SENSEX current price: 81425.15
SENSEX history (last 5 days):
                                   Open          High           Low  \
Date                                                                  
2025-09-04 00:00:00+05:30  81456.671875  81456.671875  80608.937500   
2025-09-05 00:00:00+05:30  81012.421875  81036.562500  80321.187500   
2025-09-08 00:00:00+05:30  80904.398438  81171.382812  80733.070312   
2025-09-09 00:00:00+05:30  81129.687500  81181.367188  80927.968750   
2025-09-10 00:00:00+05:30  81504.359375  81643.882812  81235.421875   

                                  Close  Volume  Dividends  Stock Splits  
Date                                                                      
2025-09-04 00:00:00+05:30  80718.007812   17000        0.0           0.0  
2025-09-05 00:00:00+05:30  80710.757812    8200        0.0           0.0  
2025-09-08 00:00:00+05:30  80787.296875    8700        0.0           0.0  
2025-09-09 0